# Create feature tables for regression
Use corrected single mutant relative fitness and genetic interaction neutrality functions (Mani 2008 PNAS) to predict the relative fitness of the double mutant

- 9 features:

  Wa (max)

  Wb (min)

  additive (Wa + Wb)

  difference (Wa - Wb)

  multiplicative (Wa * Wb)
  
  log2_mani (log2[((2^Wa) − 1)*((2^Wb) − 1) + 1])

  pslog10_additive (pseudolog10(Wa * Wb))

  pslog10_difference (pseudolog10(Wa / Wb))

  mean ((Wa + Wb)/2)

Labels:
- Wab as total seed count (TSC)

Relative fitness (W): total seed count (corrected for batch effects)

Wa: relative fitness of gene mutant A

Wb: relative fitness of gene mutant B

Wab: relative fitness of the double mutant

In [1]:
# Read in the data
import pandas as pd
# data = pd.read_csv('../ara_data/double_mutant_fitness_data_05312024_all_corrected_linear_b.txt', sep='\t')
data = pd.read_csv('../ara_data/fitness_data_for_Kenia_09232024_all_corrected_brianna.txt', sep='\t')
data

,Set,Flat,Column,Row,Number,Type,Genotype,Subline,MA,MB,...,TSC_emmean,TSC_SE,TSC_df,TSC_lower.CL,TSC_upper.CL,SH_emmean,SH_SE,SH_df,SH_lower.CL,SH_upper.CL
0,1,1,4,1,4,BORDER,MB,001-MB-2,WT,MUT,...,38.599448,3.785740,6.300279,29.442053,47.756843,-0.045469,2.161922,5.665810,-5.412067,5.321129
1,1,1,6,1,6,BORDER,DM,001-DM-2,MUT,MUT,...,40.135113,3.785609,6.351307,30.994854,49.275373,2.960672,2.061995,5.505510,-2.196761,8.118106
2,1,1,8,1,8,BORDER,MA,001-MA-2,MUT,WT,...,51.300770,3.976882,6.979477,41.891330,60.710211,4.511428,2.158732,6.262273,-0.717634,9.740490
3,1,1,10,1,10,BORDER,WT,001-WT-2,WT,WT,...,54.827980,3.758862,7.225160,45.995527,63.660432,NaN,NaN,NaN,NaN,NaN
4,1,1,6,3,26,INSIDE,MB,001-MB-2,WT,MUT,...,38.599448,3.785740,6.300279,29.442053,47.756843,-0.045469,2.161922,5.665810,-5.412067,5.321129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,813,1,6,20,196,BORDER,MB,813-MB-3,WT,MUT,...,136.769588,21.430399,1.231306,-39.988323,313.527500,1.619544,1.555047,6.727360,-2.087985,5.327073
27071,813,1,7,20,197,BORDER,DM,813-DM-1,MUT,MUT,...,135.431506,21.471827,1.243478,-38.558672,309.421684,3.950617,1.543325,8.527448,0.429659,7.471576
27072,813,1,8,20,198,BORDER,WT,813-WT-4,WT,WT,...,111.261848,21.344605,1.213022,-69.673959,292.197655,3.521429,1.517188,6.379829,-0.138073,7.180930
27073,813,1,9,20,199,BORDER,MA,813-MA-2,MUT,WT,...,121.667552,21.538422,1.257434,-49.436348,292.771452,2.268218,1.607247,8.015060,-1.436889,5.973325


## Calculate the relative fitness of the single and double mutants using the corrected trait values

In [2]:
# Subset the corrected trait values
data_avg = data.loc[:,data.columns.isin(['Set', 'Flat', 'Genotype'])\
    | (data.columns.str.endswith('_emmean'))].\
    groupby(['Set', 'Flat', 'Genotype']).mean()
data_avg

GN_emmean   PG_emmean  DTB_emmean  LN_emmean  DTF_emmean  \
Set Flat Genotype                                                             
1   1    DM         4.800409   96.015463   31.886499   8.935718   32.898779   
         MA         5.084596  101.743181   30.297541   8.278198   32.588143   
         MB         5.122958  101.741394   30.825010   8.679016   33.196756   
         WT         5.204000  104.048629   28.788941   8.153668   31.601940   
    2    DM         4.800409   96.015463   31.886499   8.935718   32.898779   
...                      ...         ...         ...        ...         ...   
815 1    WT         4.214715   69.433570   26.995461  10.078244   29.292578   
823 1    DM         4.851064   97.021277   27.600000  11.417577   29.719597   
         MA         4.520000   90.400000   26.638298  11.889593   29.170577   
         MB         4.510204   90.204082   27.020833  11.557523   29.133483   
         WT         3.880000   77.600000   26.148936  12.123635   29.291535   

                   SN_emmean     WO_emmean  FN_emmean  SPF_emmean  TSC_emmean  \
Set Flat Genotype                                                               
1   1    DM         2.920827  1.372256e+00   0.158597   13.188742   40.135113   
         MA         2.553109  8.952630e-01   0.068230   18.062058   51.300770   
         MB         2.929391  1.267982e+00   0.065808   13.128749   38.599448   
         WT         2.729105  7.402272e-01   0.088236   19.029382   54.827980   
    2    DM         2.920827  1.372256e+00   0.158597   13.188742   40.135113   
...                      ...           ...        ...         ...         ...   
815 1    WT         4.000000  4.445552e-01  10.571871   10.331973   50.057337   
823 1    DM        11.760870  3.913043e-01   2.130064   22.883859  281.904744   
         MA        12.102041  8.163265e-02   1.379226   24.999167  309.144277   
         MB        12.812500  7.083333e-01   1.503776   22.824889  298.633144   
         WT        12.600000  6.393248e-17   1.763625   25.400185  331.049474   

                   SH_emmean  
Set Flat Genotype             
1   1    DM         2.960672  
         MA         4.511428  
         MB        -0.045469  
         WT         2.728458  
    2    DM         2.960672  
...                      ...  
815 1    WT         5.107143  
823 1    DM         7.873370  
         MA         8.557326  
         MB        11.966486  
         WT        10.541964  

[700 rows x 11 columns]

In [3]:
# Calculate the relative fitness
W = data_avg.index.to_frame()
for trait in data_avg.columns:
    tmp = data_avg[trait].reset_index().pivot(index=['Set', 'Flat'], columns='Genotype', values=trait) # reshape tmp to wide
    W_tmp = tmp.apply(lambda x: x / x['WT'], axis=1) # relative fitness
    W_tmp = W_tmp.reset_index().melt(id_vars=['Set', 'Flat'], var_name='Genotype', value_name=f'W_{trait}') # reshape W_tmp back to long
    W_tmp.set_index(['Set', 'Flat', 'Genotype'], inplace=True)
    W = pd.concat([W, W_tmp], ignore_index=False, axis=1)
W

Set  Flat Genotype  W_GN_emmean  W_PG_emmean  W_DTB_emmean  \
Set Flat Genotype                                                               
1   1    DM          1     1       DM     0.922446     0.922794      1.107595   
         MA          1     1       MA     0.977055     0.977843      1.052402   
         MB          1     1       MB     0.984427     0.977825      1.070724   
         WT          1     1       WT     1.000000     1.000000      1.000000   
    2    DM          1     2       DM     0.922446     0.922794      1.107595   
...                ...   ...      ...          ...          ...           ...   
815 1    WT        815     1       WT     1.000000     1.000000      1.000000   
823 1    DM        823     1       DM     1.250274     1.250274      1.055492   
         MA        823     1       MA     1.164948     1.164948      1.018714   
         MB        823     1       MB     1.162424     1.162424      1.033344   
         WT        823     1       WT     1.000000     1.000000      1.000000   

                   W_LN_emmean  W_DTF_emmean  W_SN_emmean   W_WO_emmean  \
Set Flat Genotype                                                         
1   1    DM           1.095914      1.041037     1.070251  1.853831e+00   
         MA           1.015273      1.031207     0.935512  1.209444e+00   
         MB           1.064431      1.050466     1.073389  1.712963e+00   
         WT           1.000000      1.000000     1.000000  1.000000e+00   
    2    DM           1.095914      1.041037     1.070251  1.853831e+00   
...                        ...           ...          ...           ...   
815 1    WT           1.000000      1.000000     1.000000  1.000000e+00   
823 1    DM           0.941762      1.014614     0.933402  6.120588e+15   
         MA           0.980695      0.995871     0.960479  1.276857e+15   
         MB           0.953305      0.994604     1.016865  1.107940e+16   
         WT           1.000000      1.000000     1.000000  1.000000e+00   

                   W_FN_emmean  W_SPF_emmean  W_TSC_emmean  W_SH_emmean  
Set Flat Genotype                                                        
1   1    DM           1.797419      0.693073      0.732019     1.085108  
         MA           0.773270      0.949167      0.935668     1.653472  
         MB           0.745817      0.689920      0.704010    -0.016665  
         WT           1.000000      1.000000      1.000000     1.000000  
    2    DM           1.797419      0.693073      0.732019     1.085108  
...                        ...           ...           ...          ...  
815 1    WT           1.000000      1.000000      1.000000     1.000000  
823 1    DM           1.207776      0.900933      0.851549     0.746860  
         MA           0.782041      0.984212      0.933831     0.811739  
         MB           0.852662      0.898611      0.902080     1.135129  
         WT           1.000000      1.000000      1.000000     1.000000  

[700 rows x 14 columns]

In [4]:
print(data_avg.index.get_level_values('Set').nunique())
print(W.index.get_level_values('Set').nunique())

139
139


In [5]:
# Check which sets do not have double mutant information
for name, group in W.groupby(level='Set'):
    geno_levels = group.dropna(axis=0, how='all').index.get_level_values('Genotype').unique()
    if 'DM' not in geno_levels.values:
        print(geno_levels)
        print(f'Drop set {name}')

In [6]:
# This was done for 20240531 data. There were 137 sets, and 2 were dropped, leaving 135 sets.
# Drop sets that do not have double mutant information
# W = W.loc[~W.index.get_level_values('Set').isin(['845', '845E']),:]

In [7]:
W.drop(columns=['Set', 'Flat', 'Genotype'], inplace=True)
W

W_GN_emmean  W_PG_emmean  W_DTB_emmean  W_LN_emmean  \
Set Flat Genotype                                                        
1   1    DM           0.922446     0.922794      1.107595     1.095914   
         MA           0.977055     0.977843      1.052402     1.015273   
         MB           0.984427     0.977825      1.070724     1.064431   
         WT           1.000000     1.000000      1.000000     1.000000   
    2    DM           0.922446     0.922794      1.107595     1.095914   
...                        ...          ...           ...          ...   
815 1    WT           1.000000     1.000000      1.000000     1.000000   
823 1    DM           1.250274     1.250274      1.055492     0.941762   
         MA           1.164948     1.164948      1.018714     0.980695   
         MB           1.162424     1.162424      1.033344     0.953305   
         WT           1.000000     1.000000      1.000000     1.000000   

                   W_DTF_emmean  W_SN_emmean   W_WO_emmean  W_FN_emmean  \
Set Flat Genotype                                                         
1   1    DM            1.041037     1.070251  1.853831e+00     1.797419   
         MA            1.031207     0.935512  1.209444e+00     0.773270   
         MB            1.050466     1.073389  1.712963e+00     0.745817   
         WT            1.000000     1.000000  1.000000e+00     1.000000   
    2    DM            1.041037     1.070251  1.853831e+00     1.797419   
...                         ...          ...           ...          ...   
815 1    WT            1.000000     1.000000  1.000000e+00     1.000000   
823 1    DM            1.014614     0.933402  6.120588e+15     1.207776   
         MA            0.995871     0.960479  1.276857e+15     0.782041   
         MB            0.994604     1.016865  1.107940e+16     0.852662   
         WT            1.000000     1.000000  1.000000e+00     1.000000   

                   W_SPF_emmean  W_TSC_emmean  W_SH_emmean  
Set Flat Genotype                                           
1   1    DM            0.693073      0.732019     1.085108  
         MA            0.949167      0.935668     1.653472  
         MB            0.689920      0.704010    -0.016665  
         WT            1.000000      1.000000     1.000000  
    2    DM            0.693073      0.732019     1.085108  
...                         ...           ...          ...  
815 1    WT            1.000000      1.000000     1.000000  
823 1    DM            0.900933      0.851549     0.746860  
         MA            0.984212      0.933831     0.811739  
         MB            0.898611      0.902080     1.135129  
         WT            1.000000      1.000000     1.000000  

[700 rows x 11 columns]

## Feature tables for total seed count (TSC) and the other traits
1. Ensure the first single mutant column has the greater value than the other single mutant
2. Calculate the 6 neutrality functions using the SMF data

In [8]:
def pseudolog10(x):
  # Pseudo-Logarithm of base 10, is defined for all real numbers. Use instead of
  # log10, which returns infinite/NaN values for x <= 0
  return (np.log((x/2) + np.sqrt((x/2)**2 + 1)) / np.log(10))

In [9]:
import numpy as np
for trait in W.columns.values:
    print(trait)
    trait_df = W[trait].unstack()
    trait_df.drop(columns='WT', inplace=True) # drop wild type
    trait_df.index = trait_df.index.get_level_values('Set').astype(str) # drop flat, since values don't change for sets with flat included as the random effect
    trait_df.index = trait_df.index.get_level_values('Set').str.strip()
    trait_df = trait_df.groupby('Set').mean()
    trait_df.dropna(axis=0, how='all', inplace=True) # drop rows with all NaN values (this particular trait was not collected for these sets)

    # reorder the values in MA and MB so that MA is always greater than MB
    trait_df['MA_new'] = trait_df['MA'].where(trait_df['MA'] > trait_df['MB'], trait_df['MB'])
    trait_df['MB_new'] = trait_df['MB'].where(trait_df['MA'] > trait_df['MB'], trait_df['MA'])

    # calculate the expected double mutant relative fitness using the neutrality functions
    trait_df['mean'] = (trait_df['MA_new'] + trait_df['MB_new']) / 2
    trait_df['multiplicative'] = trait_df['MA_new'] * trait_df['MB_new']
    trait_df['additive'] = trait_df['MA_new'] + trait_df['MB_new'] - 1
    trait_df['difference'] = trait_df['MA_new'] - trait_df['MB_new']
    trait_df['log2_mani'] = np.log2(((2**trait_df['MA_new']) - 1) * ((2**trait_df['MB_new']) - 1) + 1)
    trait_df['pslog10_additive'] = trait_df.apply(lambda x: pseudolog10(x['MA_new'] * x['MB_new']), axis=1)
    trait_df['pslog10_difference'] = trait_df.apply(lambda x: pseudolog10(x['MA_new'] / x['MB_new']), axis=1)
    # pseudolog10(trait_df['MA_new'] * trait_df['MB_new'])
    # trait_df['log10_difference'] = pseudolog10(trait_df['MA_new'] / trait_df['MB_new'])

    # create sample IDs
    ID = 'Set_' + trait_df.index.astype(str)
    trait_df.insert(0, 'ID', ID)
    trait_df.set_index('ID', inplace=True)

    # save the feature table for the regression models to predict the trait using GI defs
    trait_df.rename(columns={'DM': trait}, inplace=True)
    trait_df.replace([np.inf, -np.inf], np.nan) # replace inf with NaN
    trait_df.dropna(axis=0, how='all', inplace=True) # drop rows with all NaN values
    trait_df.loc[:,[trait, 'MA_new', 'MB_new', 'mean', 'multiplicative', 
        'additive', 'difference', 'log2_mani', 'pslog10_additive', 'pslog10_difference']].\
        to_csv(f'../ara_data/1_feature_tables/{trait}_feature_table.tsv', sep='\t')
    trait_df.corr(method='pearson').to_csv(f'../ara_data/1_feature_tables/{trait}_feature_correlation.tsv', sep='\t')
    print(trait_df.shape)

W_GN_emmean
(137, 12)
W_PG_emmean
(137, 12)
W_DTB_emmean
(138, 12)
W_LN_emmean
(138, 12)
W_DTF_emmean
(138, 12)
W_SN_emmean
(139, 12)
W_WO_emmean
(139, 12)
W_FN_emmean
(139, 12)
W_SPF_emmean
(139, 12)
W_TSC_emmean
(139, 12)
W_SH_emmean
(136, 12)


/home/seguraab/miniconda3/envs/py310/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
/tmp/ipykernel_76829/2481395451.py:4: RuntimeWarning: invalid value encountered in scalar add
  return (np.log((x/2) + np.sqrt((x/2)**2 + 1)) / np.log(10))
/tmp/ipykernel_76829/1677490865.py:22: RuntimeWarning: divide by zero encountered in scalar divide
  trait_df['pslog10_difference'] = trait_df.apply(lambda x: pseudolog10(x['MA_new'] / x['MB_new']), axis=1)
/tmp/ipykernel_76829/1677490865.py:22: RuntimeWarning: invalid value encountered in scalar divide
  trait_df['pslog10_difference'] = trait_df.apply(lambda x: pseudolog10(x['MA_new'] / x['MB_new']), axis=1)
/tmp/ipykernel_76829/1677490865.py:22: RuntimeWarning: divide by zero encountered in scalar divide
  trait_df['pslog10_difference'] = trait_df.apply(lambda x: pseudolog10(x['MA_new'] / x['MB_new']), axis=1)


{trait}_feature_table.tsv and {trait}_feature_correlation.tsv were saved to ara_data/1_feature_tables/20240923_melissa_ara_data_features

## Stratified K-fold train-test split

In [10]:
from sklearn.model_selection import StratifiedKFold
# sklearn v1.2.2
np.random.seed(20240606)

for trait in W.columns.values:
    trait_df = pd.read_csv(
        f'../ara_data/1_feature_tables/20240923_melissa_ara_data_features/{trait}_feature_table.tsv',
        sep='\t', index_col=0)
    print(trait, 'trait_df', trait_df.shape)

    # Create bins
    trait_df['label_bin'] = pd.cut(trait_df[trait], bins=[-np.inf,
        np.quantile(trait_df[trait], 0.25), np.quantile(trait_df[trait], 0.5),
        np.quantile(trait_df[trait], 0.75), np.inf], labels=[0, 1, 3, 4])

    # Apply stratified k-fold train-test split
    y = trait_df['label_bin']
    X = trait_df.drop(columns=[trait, 'label_bin'], axis=1)
    skf = StratifiedKFold(n_splits=6, random_state=20240606, shuffle=True)

    i = 0
    for train_idx, test_idx, in skf.split(X, y):
        if i == 0:
            # Write test set to file
            with open(f'../ara_data/{trait}_test_instances.txt', 'w') as f:
                for ID in test_idx:
                    f.write("%s\n" % trait_df.iloc[ID,:].name)
        else:
            break
        i += 1

W_GN_emmean trait_df (137, 10)
W_PG_emmean trait_df (137, 10)
W_DTB_emmean trait_df (138, 10)
W_LN_emmean trait_df (138, 10)
W_DTF_emmean trait_df (138, 10)
W_SN_emmean trait_df (139, 10)
W_WO_emmean trait_df (139, 10)
W_FN_emmean trait_df (139, 10)
W_SPF_emmean trait_df (139, 10)
W_TSC_emmean trait_df (139, 10)
W_SH_emmean trait_df (136, 10)


{trait}_test_instsances.txt files were saved to ara_data/1_feature_tables/20240923_melissa_ara_data_features

## Run XGBoost Regression model on TSC (total seed count)

### Use shap conda environment and run on the command line
__Using all 9 neutrality functions:__
```python
python 1b_xgb_regression.py \
    -X ../ara_data/1_feature_tables/20240923_melissa_ara_data_features/W_TSC_emmean_feature_table.tsv \
    -y_name W_TSC_emmean \
    -test ../ara_data/1_feature_tables/20240923_melissa_ara_data_features/W_TSC_emmean_test_instances.txt \
    -save ../output/1_xgb_regression_ara/20240923_melissa_ara_data_results \
    -prefix W_TSC_emmean \
    -tag use_Neutrality_Funcs \
    -fold 5 -n 20 -plot t
```

__Using only the four established neutrality functions:__
```python
python 1b_xgb_regression.py \
    -X ../ara_data/1_feature_tables/20240923_melissa_ara_data_features/W_TSC_emmean_feature_table.tsv \
    -y_name W_TSC_emmean \
    -feat_list MB_new,multiplicative,additive,log2_mani \
    -test ../ara_data/1_feature_tables/20240923_melissa_ara_data_features/W_TSC_emmean_test_instances.txt \
    -save ../output/1_xgb_regression_ara/20240923_melissa_ara_data_results \
    -prefix W_TSC_emmean_established_neut_funcs \
    -tag use_Established_Neutrality_Funcs \
    -fold 5 -n 20 -plot t
```

__Using only the single mutant information:__
```python
python 1b_xgb_regression.py \
    -X ../ara_data/1_feature_tables/20240923_melissa_ara_data_features/W_TSC_emmean_feature_table.tsv \
    -y_name W_TSC_emmean \
    -feat_list MA_new,MB_new \
    -test ../ara_data/1_feature_tables/20240923_melissa_ara_data_features/W_TSC_emmean_test_instances.txt \
    -save ../output/1_xgb_regression_ara/20240923_melissa_ara_data_results \
    -prefix W_TSC_emmean_SMF_only \
    -tag use_SMF_only \
    -fold 5 -n 20 -plot t
```

### Gradient Boosting Model (Use ml-pipe1.5 conda Environment)
__Using all 9 neutrality functions:__
```python
python 1b_gb_regression.py \
    -X ../ara_data/W_TSC_emmean_feature_table.txt \
    -y_name W_TSC_emmean \
    -test ../ara_data/W_TSC_emmean_test_instances.txt \
    -save ../output/1_gb_regression_ara/ \
    -prefix W_TSC_emmean_gb \
    -tag use_SMF_and_Neutrality_Funcs \
    -fold 5 -n 10 -plot t
```

__Using only the four established neutrality functions:__
```python
```

__Using only the single mutant information:__
```python
```

### Multi-layer Perceptron
__Using all 9 neutrality functions:__


__Using only the four established neutrality functions:__

## Feature tables for predicting co-function from fitness data only

## Feature tables for predicting co-function from the GI definitions